# MATH 60210 

Nasim Siami

In [1]:
import numpy as np
import numpy.random as npr
import pandas as pd
import statsmodels.api as sm

RawData = pd.read_excel('PSet_1_H2022.xlsx',index_col=0)

## Data

The data that you'll use for this assignment is in `PSet_1_H2022.xlsx`.

This contains monthly US data from the beginning of 2001 to the end of 2021.<br>
The series are
- the S&P500 Index (`SP500`)
- the Consumer Price Index (`Inflation_CPI`)
- the price of oil (`Oil Prices`)

All series are monthly rates of change. 


---
## Question 1 - Simulation

### Part a

Create a function ``Q1_OLS3(df)`` that accepts a dataframe (`df`) with 3 series.<br>
It then runs the OLS regression 
$$
Y = \beta_0 + \beta_1 \cdot X1 + \beta_2 \cdot X2 + \epsilon
$$
where
- $Y$  is the first series in the dataframe  (df column 0)
- $X1$ is the second series in the dataframe (df column 1)
- $X2$ is the third series in the dataframe  (df column 2)

The function should return a 2x2 NumPy array
- element 0,0 contains the non-robust F-statistic testing $H_0 : \beta_1 = \beta_2 = 0$.
- element 1,0 contains the heteroscedasticity-robust F-statistic (*`HC3`*) testing $H_0 : \beta_1 = \beta_2 = 0$.
- element 0,1 contains the estimated p-value of the non-robust F-statistic.
- element 1,1 contains the estimated p-value of the robust F-statistic.

In [2]:
def Q1_OLS3(df):
    """
    This function accepts a dataframe with 3 variables [Y,X1,X2]
    It then regresses Y on a constant, X1 and X2 using OLS
    and tests the joint null that X1 and X2 are insignificant.

    It returns an 2x2 array of F-statistics and p-values 
    using both non-robust and robust (HC3) standard errors. 
    """

    ### BEGIN SOLUTION
    # Define the model
    model = sm.OLS(df.iloc[:, 0], sm.add_constant(df.iloc[:, 1:]))
    B_Restr = np.array(([0, 1, 0], [0, 0, 1]))  # restrict \beta1 = \beta2 = 0
    Results = model.fit()  # fit the model using conventional std errs
    Ftest1 = Results.f_test(B_Restr)  # calculate the F statistic
    RobResults = Results.get_robustcov_results(
        cov_type='HC3')  # calculate robust std errs
    Ftest2 = RobResults.f_test(B_Restr)  # calculate the robust F statistic
    return np.array([[Ftest1.fvalue, Ftest1.pvalue],
                     [Ftest2.fvalue, Ftest2.pvalue]])
    ### END SOLUTION


In [3]:
    
assert np.sum(Q1_OLS3(RawData) >= 0) == 4  # You should return 4 non-negative values.
    


### Part b

Create a function `Q1_ShuffleY(df)` that accepts a dataframe \(`df`\) with 3 series.<br>  

The function should return a dataframe of the same dimension whose only difference is that the values of the series in the first column have been randomly re\-ordered.

Your function should not seed the RNG.



In [4]:
def Q1_ShuffleY(df):
    """
    This function accepts a dataframe with 3 variables [Y,X1,X2]
    It randomly changes the order of the values in Y.
    It returns the changed dataframe.
    """
    ### BEGIN SOLUTION
    NewY = df.copy()
    NewY.iloc[:, 0] = npr.permutation(NewY.iloc[:, 0])
    return NewY
    ### END SOLUTION

In [5]:
# 2nd and 3rd series should not change.
assert np.array_equal(Q1_ShuffleY(RawData).iloc[:, 1:], RawData.iloc[:, 1:])
# Descriptive statistics of first series should not change.
assert Q1_ShuffleY(RawData).iloc[:, 0].max(0)  == RawData.iloc[:, 0].max(0)
assert Q1_ShuffleY(RawData).iloc[:, 0].min(0)  == RawData.iloc[:, 0].min(0)
assert abs(Q1_ShuffleY(RawData).iloc[:, 0].mean(0) - RawData.iloc[:, 0].mean(0)) < 1E-6
assert abs(Q1_ShuffleY(RawData).iloc[:, 0].std(0)  - RawData.iloc[:, 0].std(0) ) < 1E-6
    
  

### Part c

Create a function ``Q1_BootstrapY(df)`` that accepts a dataframe (`df`) with 3 series.<br>
The function should return a dataframe of the same dimension whose only difference is that the values of the series in the first column have been randomly re-sampled (with replacement.) 

Your function should not seed the RNG.


In [6]:
def Q1_BootstrapY(df):
    """
    This function accepts a dataframe with 3 variables [Y,X1,X2]
    It replaces Y with bootstrapped (randomly drawn, with replacement) values of Y. 
    It returns the changed dataframe.
    """
    ### BEGIN SOLUTION
    NewY = df.copy()
    NewY.iloc[:, 0] = npr.choice(NewY.iloc[:, 0],NewY.shape[0])
    return NewY
    ### END SOLUTION


In [7]:
# 2nd and 3rd series should not change.
assert np.array_equal(Q1_BootstrapY(RawData).iloc[:, 1:], RawData.iloc[:, 1:])

   

### Part d

Create a function `Q1_Simulate(df,F0,N,Replace,Robust)` that accepts  


- a dataframe (`df`) with 3 series, and
- a real scalar (`F0`)
- an integer > 0 (`N`)
- two Booleans (`Robust, Replace`)

The function then

- resamples the values in the first column of the dataframe
  - If `Replace == True` then it uses `Q1_BootstrapY()` to do so.
  - Otherwise, it uses `Q1_ShuffleY()` to do so.
- runs `Q1_OLS3()` on the resampled data
  - it calculates the conventional F-statistic and its p-value if `Robust == False`.
  - otherwise it calculates the heteroscedasticity-robust F-statistic and its p-value.
- repeats the resampling and regression steps `N` times.

The function should return the fraction of times that the F-statistics it has calculated are greater than `F0`.<br>
(It should not seed the RNG.)



In [8]:
def Q1_Simulate(df, F0, N, Replace, Robust):
    """
    This function accepts 
    - a dataframe (df) with 3 variables [Y,X1,X2]
    - a real scalar (F0)
    - an integer (N)
    - two Booleans (Robust, Replace)
    
    It then resamples the values in Y, regresses Y on a constant, X1 and X2, testing the null hypothesis that the coefficients on X1 and X2 are both 0. This is repeated N times. 
    
    If Replace == True, Y is resampled with replacement.
    If Robust == True, robust (HC3) test statistics are calculated instead of non-robust.
    
    The function returns the fraction of times that the test statistics calculated are greater than F0.
    """
    ### BEGIN SOLUTION
    NoBigger = 0
    # start the loop
    for j in range(N):
        if Replace:
            df = Q1_BootstrapY(df)  # Bootstrap Y
        else:
            df = Q1_ShuffleY(df)  # Shuffle Y
        # run regression and get F-stats and p-values
        F_out = Q1_OLS3(df)
        if Robust and (F_out[1, 0] > F0):
            NoBigger += 1
        elif (not Robust) and (F_out[0, 0] > F0):
            NoBigger += 1
    return NoBigger / N
    ### END SOLUTION


In [9]:
# Does it give plausible results for random data?    
test0 = Q1_Simulate(pd.DataFrame(npr.normal(size=(1000,3))),3,100,True,True) 
test1 = Q1_Simulate(pd.DataFrame(npr.normal(size=(1000,3))),3,100,False,False) 
assert (test0 < 0.15) and (test0 >= 0)
assert (test1 < 0.15) and (test1 >= 0)


### Part e

Using the data in `PSet_1_H2022.xlsx`, regress `SP500` on a constant, 
`Inflation_CPI` and `Oil Prices`. 

- Test the $H_0$ that the coefficients on `Inflation_CPI` and `Oil Prices` are both zero.
- Print your test statistic and its estimated p-value.

Now use your answer in (d) above to examine whether your conclusions from that test are reliable. 

- **Suggestion:** In (d), avoid using a value of `N` $ \lt 500 .$.
- If you didn't manage to get (d) working, explain how you would use its results to answer these questions.
Briefly answer the following questions. (4 points each)



In [11]:
#
# First, let's see what we get using `RawData`
#
Results = sm.OLS(RawData.iloc[:, 0], sm.add_constant(RawData.iloc[:, 1:])).fit()
print(Results.summary())
B_Restr = np.array(([0, 1, 0], [0, 0, 1]))  # restrict \beta1 = \beta2 = 0
print(f'Conventional F-test: {Results.f_test(B_Restr)}')
RobResults = Results.get_robustcov_results(
    cov_type='HC3')  # calculate robust std errs
print(f'Robust F-test: {RobResults.f_test(B_Restr)}') 
#
# Now let's compare those p-values for the robust and non-robust tests to what we get 
# on data simulated under H0
#
n_trials = 2000
print(f'Under H0, the non-robust F-statistic has a simulated p-value of ')
print(f'      {Q1_Simulate(RawData, 5.473, n_trials, Replace=False, Robust=False)} with Approximate Randomization,')
print(f'  and {Q1_Simulate(RawData, 5.473, n_trials, Replace=True, Robust=False)} with a Bootstrap.')

print(f'Under H0, the robust F-statistic has a simulated p-value of ')
print(f'      {Q1_Simulate(RawData, 2.544, n_trials, Replace=False, Robust=True)} with Approximate Randomization,')
print(f'  and {Q1_Simulate(RawData, 2.544, n_trials, Replace=True, Robust=True)} with a Bootstrap.')

                            OLS Regression Results                            
Dep. Variable:                  SP500   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     5.473
Date:                Sun, 13 Mar 2022   Prob (F-statistic):            0.00472
Time:                        16:39:03   Log-Likelihood:                 441.32
No. Observations:                 252   AIC:                            -876.6
Df Residuals:                     249   BIC:                            -866.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0084      0.003      2.791

      0.009 with Approximate Randomization,


  and 0.0035 with a Bootstrap.
Under H0, the robust F-statistic has a simulated p-value of 


      0.088 with Approximate Randomization,


  and 0.012 with a Bootstrap.


**1\)** Does the non\-robust F\-test give reliable results on our data?  



Yes it does. 

It reports a p-value of roughly 0.005 which is very close to simulated values.

**2)** Does the robust F-test give reliable results on our data?

It's hard to say.

The Approximate Randomization gives a similar p-value, but the Bootstrap may suggest a much lower value. 

**3)** What do the above simulations tell us about the power of our F-tests?

They tell us *nothing* about the power of the test because we are simulating under $H_0$.